In [3]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate, KFold, GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import os

In [7]:
os.chdir('/Users/dhanushkikkisetti/Documents/Data Hackathon')

In [27]:
data=pd.read_csv("insurance_claims.csv")
#data link:https://github.com/RumanaSk/Fraud-Detection-in-Insurance/blob/master/insurance_claims.csv 

In [28]:
data.head()

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported
0,328,48,521585,17-10-2014,OH,250/500,1000,1406.91,0,466132,...,2,YES,71610,6510,13020,52080,Saab,92x,2004,Y
1,228,42,342868,27-06-2006,IN,250/500,2000,1197.22,5000000,468176,...,0,?,5070,780,780,3510,Mercedes,E400,2007,Y
2,134,29,687698,06-09-2000,OH,100/300,2000,1413.14,5000000,430632,...,3,NO,34650,7700,3850,23100,Dodge,RAM,2007,N
3,256,41,227811,25-05-1990,IL,250/500,2000,1415.74,6000000,608117,...,2,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,Y
4,228,44,367455,06-06-2014,IL,500/1000,1000,1583.91,6000000,610706,...,1,NO,6500,1300,650,4550,Accura,RSX,2009,N


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 39 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   months_as_customer           1000 non-null   int64  
 1   age                          1000 non-null   int64  
 2   policy_number                1000 non-null   int64  
 3   policy_bind_date             1000 non-null   object 
 4   policy_state                 1000 non-null   object 
 5   policy_csl                   1000 non-null   object 
 6   policy_deductable            1000 non-null   int64  
 7   policy_annual_premium        1000 non-null   float64
 8   umbrella_limit               1000 non-null   int64  
 9   insured_zip                  1000 non-null   int64  
 10  insured_sex                  1000 non-null   object 
 11  insured_education_level      1000 non-null   object 
 12  insured_occupation           1000 non-null   object 
 13  insured_hobbies    

In [30]:
data=data.replace('?',np.nan)

In [31]:
check=pd.DataFrame(data=data.isnull().sum(),columns=["Null Values"])
check['perenctage']=check['Null Values'].apply(lambda x: x/data.shape[0])
check

,Null Values,perenctage
months_as_customer,0,0.000
age,0,0.000
policy_number,0,0.000
policy_bind_date,0,0.000
policy_state,0,0.000
policy_csl,0,0.000
policy_deductable,0,0.000
policy_annual_premium,0,0.000
umbrella_limit,0,0.000
insured_zip,0,0.000


In [32]:
data=data.drop(['police_report_available','property_damage','collision_type','policy_number','policy_bind_date'
           ,'incident_date','incident_city','incident_location','auto_model'],axis=1)

In [34]:
data

,months_as_customer,age,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_sex,insured_education_level,...,number_of_vehicles_involved,bodily_injuries,witnesses,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_year,fraud_reported
0,328,48,OH,250/500,1000,1406.91,0,466132,MALE,MD,...,1,1,2,71610,6510,13020,52080,Saab,2004,Y
1,228,42,IN,250/500,2000,1197.22,5000000,468176,MALE,MD,...,1,0,0,5070,780,780,3510,Mercedes,2007,Y
2,134,29,OH,100/300,2000,1413.14,5000000,430632,FEMALE,PhD,...,3,2,3,34650,7700,3850,23100,Dodge,2007,N
3,256,41,IL,250/500,2000,1415.74,6000000,608117,FEMALE,PhD,...,1,1,2,63400,6340,6340,50720,Chevrolet,2014,Y
4,228,44,IL,500/1000,1000,1583.91,6000000,610706,MALE,Associate,...,1,0,1,6500,1300,650,4550,Accura,2009,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3,38,OH,500/1000,1000,1310.80,0,431289,FEMALE,Masters,...,1,0,1,87200,17440,8720,61040,Honda,2006,N
996,285,41,IL,100/300,1000,1436.79,0,608177,FEMALE,PhD,...,1,2,3,108480,18080,18080,72320,Volkswagen,2015,N
997,130,34,OH,250/500,500,1383.49,3000000,442797,FEMALE,Masters,...,3,2,3,67500,7500,7500,52500,Suburu,1996,N
998,458,62,IL,500/1000,2000,1356.92,5000000,441714,MALE,Associate,...,1,0,1,46980,5220,5220,36540,Audi,1998,N


In [33]:
object_type=[data.dtypes.index[i] for i in range(data.shape[1]) if data.dtypes[i]=='object']
for column in object_type:
    print(f"The variables in {column} \n")
    print(data[column].value_counts(),"\n")


The variables in policy_state 

OH    352
IL    338
IN    310
Name: policy_state, dtype: int64 

The variables in policy_csl 

250/500     351
100/300     349
500/1000    300
Name: policy_csl, dtype: int64 

The variables in insured_sex 

FEMALE    537
MALE      463
Name: insured_sex, dtype: int64 

The variables in insured_education_level 

JD             161
High School    160
Associate      145
MD             144
Masters        143
PhD            125
College        122
Name: insured_education_level, dtype: int64 

The variables in insured_occupation 

machine-op-inspct    93
prof-specialty       85
tech-support         78
sales                76
exec-managerial      76
craft-repair         74
transport-moving     72
other-service        71
priv-house-serv      71
armed-forces         69
adm-clerical         65
protective-serv      63
handlers-cleaners    54
farming-fishing      53
Name: insured_occupation, dtype: int64 

The variables in insured_hobbies 

reading           64
exerci

In [35]:
data['policy_state']=data['policy_state'].map({'OH':0,'IL':1,'IN':2})
data['policy_csl']=data['policy_csl'].map({'250/500':0,'100/300':1,'500/1000':2})
data['insured_sex']=data['insured_sex'].map({'FEMALE':0,'MALE':1})
data['incident_type']=data['incident_type'].map({'Multi-vehicle Collision':0,'Single Vehicle Collision':1
                                                 ,'Vehicle Theft':2,'Parked Car':3})
data['incident_severity']=data['incident_severity'].map({'Minor Damage':0,'Total Loss':1,'Major Damage':2,'Trivial Damage':3})
data['authorities_contacted']=data['authorities_contacted'].map({'Police':0,'Fire':1,'Other':2,'Ambulance':3,'None':4})
data['fraud_reported']=data['fraud_reported'].map({'N':0,'Y':1})

In [41]:
le=LabelEncoder()
object_type=[data.dtypes.index[i] for i in range(data.shape[1]) if data.dtypes[i]=='object']
for column in object_type:
    data[column]=le.fit_transform(data[column])


,months_as_customer,age,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_sex,insured_education_level,...,number_of_vehicles_involved,bodily_injuries,witnesses,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_year,fraud_reported
0,328,48,0,0,1000,1406.91,0,466132,1,4,...,1,1,2,71610,6510,13020,52080,10,2004,1
1,228,42,2,0,2000,1197.22,5000000,468176,1,4,...,1,0,0,5070,780,780,3510,8,2007,1
2,134,29,0,1,2000,1413.14,5000000,430632,0,6,...,3,2,3,34650,7700,3850,23100,4,2007,0
3,256,41,1,0,2000,1415.74,6000000,608117,0,6,...,1,1,2,63400,6340,6340,50720,3,2014,1
4,228,44,1,2,1000,1583.91,6000000,610706,1,0,...,1,0,1,6500,1300,650,4550,0,2009,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3,38,0,2,1000,1310.80,0,431289,0,5,...,1,0,1,87200,17440,8720,61040,6,2006,0
996,285,41,1,1,1000,1436.79,0,608177,0,6,...,1,2,3,108480,18080,18080,72320,13,2015,0
997,130,34,0,0,500,1383.49,3000000,442797,0,5,...,3,2,3,67500,7500,7500,52500,11,1996,0
998,458,62,1,2,2000,1356.92,5000000,441714,1,0,...,1,0,1,46980,5220,5220,36540,1,1998,0


In [43]:
print(data.head())
data.dtypes

   months_as_customer  age  policy_state  policy_csl  policy_deductable  \
0                 328   48             0           0               1000   
1                 228   42             2           0               2000   
2                 134   29             0           1               2000   
3                 256   41             1           0               2000   
4                 228   44             1           2               1000   

   policy_annual_premium  umbrella_limit  insured_zip  insured_sex  \
0                1406.91               0       466132            1   
1                1197.22         5000000       468176            1   
2                1413.14         5000000       430632            0   
3                1415.74         6000000       608117            0   
4                1583.91         6000000       610706            1   

   insured_education_level  ...  number_of_vehicles_involved  bodily_injuries  \
0                        4  ...                

months_as_customer               int64
age                              int64
policy_state                     int64
policy_csl                       int64
policy_deductable                int64
policy_annual_premium          float64
umbrella_limit                   int64
insured_zip                      int64
insured_sex                      int64
insured_education_level          int64
insured_occupation               int64
insured_hobbies                  int64
insured_relationship             int64
capital-gains                    int64
capital-loss                     int64
incident_type                    int64
incident_severity                int64
authorities_contacted            int64
incident_state                   int64
incident_hour_of_the_day         int64
number_of_vehicles_involved      int64
bodily_injuries                  int64
witnesses                        int64
total_claim_amount               int64
injury_claim                     int64
property_claim           

## Training Machine Learning Models

In [47]:
X=data.iloc[:,:-1]
Y=data['fraud_reported']

In [62]:
cv=KFold(n_splits=6,random_state=1,shuffle=True)
kernel = ['rbf', 'poly', 'sigmoid']
C = [0.01, 0.1, 1, 10]
params={
    'kernel':kernel,
    'C':C
}



In [63]:
grid_cv=GridSearchCV(SVC(),cv=cv,param_grid=params)
grid_cv.fit(X,Y)

GridSearchCV(cv=KFold(n_splits=6, random_state=1, shuffle=True),
             estimator=SVC(),
             param_grid={'C': [0.01, 0.1, 1, 10],
                         'kernel': ['rbf', 'poly', 'sigmoid']})

In [64]:
grid_df=pd.DataFrame(grid_cv.cv_results_)
grid_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,mean_test_score,std_test_score,rank_test_score
0,0.011498,0.002974,0.004659,0.000351,0.01,rbf,"{'C': 0.01, 'kernel': 'rbf'}",0.706587,0.760479,0.754491,0.748503,0.771084,0.777108,0.753042,0.022879,1
1,0.013485,0.000924,0.002093,0.000025,0.01,poly,"{'C': 0.01, 'kernel': 'poly'}",0.706587,0.760479,0.754491,0.748503,0.771084,0.777108,0.753042,0.022879,1
2,0.010354,0.000232,0.002227,0.000025,0.01,sigmoid,"{'C': 0.01, 'kernel': 'sigmoid'}",0.706587,0.760479,0.754491,0.748503,0.771084,0.777108,0.753042,0.022879,1
3,0.009912,0.000231,0.004481,0.000075,0.1,rbf,"{'C': 0.1, 'kernel': 'rbf'}",0.706587,0.760479,0.754491,0.748503,0.771084,0.777108,0.753042,0.022879,1
4,0.026212,0.008474,0.002294,0.000172,0.1,poly,"{'C': 0.1, 'kernel': 'poly'}",0.706587,0.760479,0.754491,0.748503,0.771084,0.777108,0.753042,0.022879,1
5,0.010800,0.000190,0.002424,0.000114,0.1,sigmoid,"{'C': 0.1, 'kernel': 'sigmoid'}",0.706587,0.760479,0.754491,0.748503,0.771084,0.777108,0.753042,0.022879,1
6,0.012014,0.000886,0.004805,0.000144,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.706587,0.760479,0.754491,0.748503,0.771084,0.777108,0.753042,0.022879,1
7,0.320253,0.143182,0.002910,0.000281,1,poly,"{'C': 1, 'kernel': 'poly'}",0.706587,0.760479,0.754491,0.748503,0.771084,0.777108,0.753042,0.022879,1
8,0.010665,0.000234,0.002258,0.000038,1,sigmoid,"{'C': 1, 'kernel': 'sigmoid'}",0.706587,0.754491,0.754491,0.748503,0.765060,0.771084,0.750036,0.020811,11
9,0.016942,0.002472,0.005337,0.000289,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.706587,0.754491,0.754491,0.748503,0.771084,0.777108,0.752044,0.022663,10
